In [6]:
import pandas as pd
import numpy as np
import random
import os

# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Update this to your exact filename
INPUT_FILE = '../data/public_emdat_custom_request_2018-2024.xlsx' 
OUTPUT_PATH = '../data/cleaned_data_final.csv'

# Column Mapping: "Exact Name in Excel" -> "New Standard Name"
# Note: EM-DAT uses 'Start Year' and 'Total Damage, Adjusted'
COLUMN_MAPPING = {
    'Disaster Type': 'disaster_type',
    'Country': 'country',
    'Start Year': 'year',
    'Start Month': 'month',
    'Total Deaths': 'casualties',
    # We prefer Adjusted damage for inflation correction
    "Total Damage, Adjusted ('000 US$)": 'economic_loss_usd', 
    'Latitude': 'latitude',
    'Longitude': 'longitude'
}
print("Configuration loaded.")

Configuration loaded.


In [8]:
# ==========================================
# 2. ENHANCED COORDINATE MAPPING (EM-DAT Specific)
# ==========================================
COUNTRY_COORDS = {
    'Afghanistan': (33.9391, 67.7100),
    'Albania': (41.1533, 20.1683),
    'Algeria': (28.0339, 1.6596),
    'Angola': (-11.2027, 17.8739),
    'Argentina': (-38.4161, -63.6167),
    'Armenia': (40.0691, 45.0382),
    'Australia': (-25.2744, 133.7751),
    'Austria': (47.5162, 14.5501),
    'Azerbaijan': (40.1431, 47.5769),
    'Bangladesh': (23.6850, 90.3563),
    'Belgium': (50.5039, 4.4699),
    'Benin': (9.3077, 2.3158),
    'Bolivia (Plurinational State of)': (-16.2902, -63.5887),
    'Bosnia and Herzegovina': (43.9159, 17.6791),
    'Botswana': (-22.3285, 24.6849),
    'Brazil': (-14.2350, -51.9253),
    'Bulgaria': (42.7339, 25.4858),
    'Burkina Faso': (12.2383, -1.5616),
    'Burundi': (-3.3731, 29.9189),
    'Cabo Verde': (16.0021, -24.0131),
    'Cambodia': (12.5657, 104.9910),
    'Cameroon': (7.3697, 12.3547),
    'Canada': (56.1304, -106.3468),
    'Central African Republic': (6.6111, 20.9394),
    'Chad': (15.4542, 18.7322),
    'Chile': (-35.6751, -71.5430),
    'China': (35.8617, 104.1954),
    'Colombia': (4.5709, -74.2973),
    'Comoros': (-11.6455, 43.3333),
    'Congo': (-0.2280, 15.8277),
    'Costa Rica': (9.7489, -83.7534),
    'Croatia': (45.1000, 15.2000),
    'Cuba': (21.5218, -77.7812),
    'Cyprus': (35.1264, 33.4299),
    'Czechia': (49.8175, 15.4730),
    'Côte d’Ivoire': (7.5400, -5.5471),
    'Democratic Republic of the Congo': (-4.0383, 21.7587),
    'Denmark': (56.2639, 9.5018),
    'Djibouti': (11.8251, 42.5903),
    'Dominican Republic': (18.7357, -70.1627),
    'Ecuador': (-1.8312, -78.1834),
    'Egypt': (26.8206, 30.8025),
    'El Salvador': (13.7942, -88.8965),
    'Equatorial Guinea': (1.6508, 10.2679),
    'Eritrea': (15.1794, 39.7823),
    'Estonia': (58.5953, 25.0136),
    'Eswatini': (-26.5225, 31.4659),
    'Ethiopia': (9.1450, 40.4897),
    'Fiji': (-17.7134, 178.0650),
    'Finland': (61.9241, 25.7482),
    'France': (46.2276, 2.2137),
    'Gabon': (-0.8037, 11.6094),
    'Gambia': (13.4432, -15.3101),
    'Georgia': (42.3154, 43.3569),
    'Germany': (51.1657, 10.4515),
    'Ghana': (7.9465, -1.0232),
    'Greece': (39.0742, 21.8243),
    'Grenada': (12.1165, -61.6790),
    'Guatemala': (15.7835, -90.2308),
    'Guinea': (9.9456, -9.6966),
    'Guinea-Bissau': (11.8037, -15.1804),
    'Guyana': (4.8604, -58.9302),
    'Haiti': (18.9712, -72.2852),
    'Honduras': (15.2000, -86.2419),
    'Hungary': (47.1625, 19.5033),
    'Iceland': (64.9631, -19.0208),
    'India': (20.5937, 78.9629),
    'Indonesia': (-0.7893, 113.9213),
    'Iran (Islamic Republic of)': (32.4279, 53.6880),
    'Iraq': (33.2232, 43.6793),
    'Ireland': (53.1424, -7.6921),
    'Israel': (31.0461, 34.8516),
    'Italy': (41.8719, 12.5674),
    'Jamaica': (18.1096, -77.2975),
    'Japan': (36.2048, 138.2529),
    'Jordan': (30.5852, 36.2384),
    'Kazakhstan': (48.0196, 66.9237),
    'Kenya': (-0.0236, 37.9062),
    'Kuwait': (29.3117, 47.4818),
    'Kyrgyzstan': (41.2044, 74.7661),
    "Lao People's Democratic Republic": (19.8563, 102.4955),
    'Latvia': (56.8796, 24.6032),
    'Lebanon': (33.8547, 35.8623),
    'Lesotho': (-29.6099, 28.2336),
    'Liberia': (6.4281, -9.4295),
    'Libya': (26.3351, 17.2283),
    'Lithuania': (55.1694, 23.8813),
    'Luxembourg': (49.8153, 6.1296),
    'Madagascar': (-18.7669, 46.8691),
    'Malawi': (-13.2543, 34.3015),
    'Malaysia': (4.2105, 101.9758),
    'Maldives': (3.2028, 73.2207),
    'Mali': (17.5707, -3.9962),
    'Malta': (35.9375, 14.3754),
    'Mauritania': (21.0079, -10.9408),
    'Mauritius': (-20.3484, 57.5522),
    'Mexico': (23.6345, -102.5528),
    'Mongolia': (46.8625, 103.8467),
    'Montenegro': (42.7087, 19.3744),
    'Morocco': (31.7917, -7.0926),
    'Mozambique': (-18.6657, 35.5296),
    'Myanmar': (21.9162, 95.9560),
    'Namibia': (-22.9576, 18.4904),
    'Nepal': (28.3949, 84.1240),
    'Netherlands (Kingdom of the)': (52.1326, 5.2913),
    'New Zealand': (-40.9006, 174.8860),
    'Nicaragua': (12.8654, -85.2072),
    'Niger': (17.6078, 8.0817),
    'Nigeria': (9.0820, 8.6753),
    'North Macedonia': (41.6086, 21.7453),
    'Norway': (60.4720, 8.4689),
    'Oman': (21.5126, 55.9233),
    'Pakistan': (30.3753, 69.3451),
    'Panama': (8.5380, -80.7821),
    'Papua New Guinea': (-6.3150, 143.9555),
    'Paraguay': (-23.4425, -58.4438),
    'Peru': (-9.1900, -75.0152),
    'Philippines': (12.8797, 121.7740),
    'Poland': (51.9194, 19.1451),
    'Portugal': (39.3999, -8.2245),
    'Qatar': (25.3548, 51.1839),
    'Republic of Korea': (35.9078, 127.7669),
    'Republic of Moldova': (47.4116, 28.3699),
    'Romania': (45.9432, 24.9668),
    'Russian Federation': (61.5240, 105.3188),
    'Rwanda': (-1.9403, 29.8739),
    'Saint Lucia': (13.9094, -60.9789),
    'Samoa': (-13.7590, -172.1046),
    'Saudi Arabia': (23.8859, 45.0792),
    'Senegal': (14.4974, -14.4524),
    'Serbia': (44.0165, 21.0059),
    'Sierra Leone': (8.4606, -11.7799),
    'Slovakia': (48.6690, 19.6990),
    'Slovenia': (46.1512, 14.9955),
    'Solomon Islands': (-9.6457, 160.1562),
    'Somalia': (5.1521, 46.1996),
    'South Africa': (-30.5595, 22.9375),
    'South Sudan': (6.8770, 31.3070),
    'Spain': (40.4637, -3.7492),
    'Sri Lanka': (7.8731, 80.7718),
    'Sudan': (12.8628, 30.2176),
    'Suriname': (3.9193, -56.0278),
    'Sweden': (60.1282, 18.6435),
    'Switzerland': (46.8182, 8.2275),
    'Syrian Arab Republic': (34.8021, 38.9968),
    'Tajikistan': (38.8610, 71.2761),
    'Thailand': (15.8700, 100.9925),
    'Timor-Leste': (-8.8742, 125.7275),
    'Togo': (8.6195, 0.8248),
    'Tonga': (-21.1790, -175.1982),
    'Trinidad and Tobago': (10.6918, -61.2225),
    'Tunisia': (33.8869, 9.5375),
    'Turkey': (38.9637, 35.2433),
    'Türkiye': (38.9637, 35.2433),
    'Uganda': (1.3733, 32.2903),
    'Ukraine': (48.3794, 31.1656),
    'United Arab Emirates': (23.4241, 53.8478),
    'United Kingdom of Great Britain and Northern Ireland': (55.3781, -3.4360),
    'United Republic of Tanzania': (-6.3690, 34.8888),
    'United States of America': (37.0902, -95.7129),
    'Uruguay': (-32.5228, -55.7658),
    'Uzbekistan': (41.3775, 64.5853),
    'Vanuatu': (-15.3767, 166.9592),
    'Venezuela (Bolivarian Republic of)': (6.4238, -66.5897),
    'Viet Nam': (14.0583, 108.2772),
    'Yemen': (15.5527, 48.5164),
    'Zambia': (-13.1339, 27.8493),
    'Zimbabwe': (-19.0154, 29.1549),
    'Taiwan (Province of China)': (23.6978, 120.9605),
    'China, Hong Kong Special Administrative Region': (22.3193, 114.1694),
    "Democratic People's Republic of Korea": (40.3399, 127.5101),
    'Sao Tome and Principe': (0.1864, 6.6131),
    'Bahamas': (25.0343, -77.3963),
    'Guadeloupe': (16.2650, -61.5510),
    'State of Palestine': (31.9522, 35.2332),
    'Puerto Rico': (18.2208, -66.5901),
    'French Guiana': (3.9339, -53.1258),
    'Saint Vincent and the Grenadines': (12.9843, -61.2872),
    'Northern Mariana Islands': (17.3308, 145.3847),
    'Belarus': (53.7098, 27.9534),
    'Curaçao': (12.1696, -68.9900),
    'Kiribati': (-3.3704, -168.7340),
    'Marshall Islands': (7.1315, 171.1845),
    'Palau': (7.5150, 134.5825),
    'Tuvalu': (-7.1095, 177.6493),
    'Mayotte': (-12.8275, 45.1662),
    'Réunion': (-21.1151, 55.5364),
    'Canary Islands': (28.2916, -16.6291),
    'Belize': (17.1899, -88.4976),
    'Guam': (13.4443, 144.7937),
    'Barbados': (13.1939, -59.5432),
    'Liechtenstein': (47.1660, 9.5554),
    'Seychelles': (-4.6796, 55.4920),
    'Micronesia (Federated States of)': (7.4256, 150.5508),
    'New Caledonia': (-20.9043, 165.6180),
    'Bhutan': (27.5142, 90.4336)
}

def get_coords_with_jitter(country_name, existing_lat, existing_lon):
    """
    Returns (lat, lon). 
    1. If existing lat/lon is valid, returns them.
    2. If missing, looks up country in dictionary.
    3. Adds slight random jitter to prevent point stacking.
    """
    
    # 1. Check if data already has coordinates
    if pd.notnull(existing_lat) and pd.notnull(existing_lon):
        return existing_lat, existing_lon
    
    # 2. Look up country
    # Try direct match
    coords = COUNTRY_COORDS.get(country_name)
    
    # 3. If found, add jitter
    if coords:
        base_lat, base_lon = coords
        # Jitter: Random float between -1.5 and +1.5 degrees
        lat_jitter = random.uniform(-1.5, 1.5)
        lon_jitter = random.uniform(-1.5, 1.5)
        return base_lat + lat_jitter, base_lon + lon_jitter
    
    # 4. If not found, return NaN (we will drop these later or handle them)
    return np.nan, np.nan

print("Coordinate mapping logic initialized.")

Coordinate mapping logic initialized.


In [9]:
# ==========================================
# 3. DATA LOADING & PROCESSING
# ==========================================
print("Loading data...")

try:
    df = pd.read_excel(INPUT_FILE)
except FileNotFoundError:
    print(f"Error: {INPUT_FILE} not found. Please ensure the file is in the folder.")
    raise

# Strip whitespace from column names to avoid ' Start Year' vs 'Start Year' issues
df.columns = df.columns.str.strip()

print(f"Initial shape: {df.shape}")

# Rename Columns
# We use a loop specifically to avoid errors if a column is slightly different
rename_dict = {}
for original_col, new_col in COLUMN_MAPPING.items():
    if original_col in df.columns:
        rename_dict[original_col] = new_col
    else:
        print(f"Warning: Column '{original_col}' not found in Excel. Checking alternatives...")

df = df.rename(columns=rename_dict)

# Verify required columns exist, if not create empty ones
required_cols = ['disaster_type', 'country', 'year', 'month', 'casualties', 'economic_loss_usd', 'latitude', 'longitude']
for col in required_cols:
    if col not in df.columns:
        print(f"Creating missing column: {col}")
        df[col] = np.nan
        
print("Data loaded and columns standardized.")

Loading data...
Initial shape: (4014, 47)
Data loaded and columns standardized.


In [10]:
# ==========================================
# 4. CLEANING & IMPUTATION
# ==========================================
print("Cleaning data...")

# Handle Missing Numerical Impact Data
# For visualization, NaN in casualties usually implies 0 or Unknown.
df['casualties'] = df['casualties'].fillna(0).astype(int)
df['economic_loss_usd'] = df['economic_loss_usd'].fillna(0).astype(float)
df['year'] = df['year'].fillna(0).astype(int)

# Clean Text Data
df['country'] = df['country'].astype(str).str.strip()
df['disaster_type'] = df['disaster_type'].astype(str).str.strip()

# Apply Coordinate Logic
print("Generating coordinates for missing locations...")
# Apply the function row by row
df[['latitude', 'longitude']] = df.apply(
    lambda row: pd.Series(get_coords_with_jitter(row['country'], row['latitude'], row['longitude'])), 
    axis=1
)

# Report on missing coordinates
missing_coords = df[df['latitude'].isna()]['country'].unique()
if len(missing_coords) > 0:
    print(f"\n[ATTENTION] The following countries still need coordinates:\n{missing_coords}")
else:
    print("\nAll countries successfully mapped!")

# Filter Invalid Years
df = df[df['year'] > 0]
print("Data cleaning complete.")

Cleaning data...
Generating coordinates for missing locations...

All countries successfully mapped!
Data cleaning complete.


In [11]:
# ==========================================
# 5. FINAL EXPORT
# ==========================================

# Select only the columns we need for the dashboard to keep file size down
final_cols = ['year', 'month', 'disaster_type', 'country', 'latitude', 'longitude', 'casualties', 'economic_loss_usd']
df_final = df[final_cols]

print(f"\nFinal shape: {df_final.shape}")
print(df_final.head())

df_final.to_csv(OUTPUT_PATH, index=False)
print(f"\nSuccess! Cleaned data saved to: {OUTPUT_PATH}")


Final shape: (4014, 8)
   year  month disaster_type                   country   latitude   longitude  \
0  2018    2.0         Flood                    Brazil -22.479000  -44.095000   
1  2022   11.0         Flood                    Rwanda  -3.262931   29.517901   
2  2024    1.0       Drought  United States of America  36.913668  -96.307373   
3  2022    8.0          Road                 Indonesia  -0.886742  113.391069   
4  2022    9.0         Water                   Tunisia  34.188661   10.957400   

   casualties  economic_loss_usd  
0           4            12492.0  
1           3                0.0  
2           0          5400000.0  
3          10                0.0  
4          20                0.0  

Success! Cleaned data saved to: ../data/cleaned_data_final.csv
